# THIS IS FOR MARKING OUT WINDOWS IN PRE-GENERATED PLANS SO THAT WE CAN GIVE THE FUNCTION MAP TO A NETWORK AND IT WILL PLACE THE WINDOWS
# TO DO - MARK WINDOWS IN A NEW WAY, THE RED LINES DOESNT WORK, MAYBE MARK THE WALLS INSTEAD?

In [ ]:
import cv2
import os
import time
import glob
import numpy as np
from scipy import misc
import PIL.Image
import io
import array
import re
import pathlib

print('running')

data_dir = pathlib.Path('E:/DataSets/cubicasa5kv1.1')
data_dir_str = 'E:/DataSets/cubicasa5kv1.1'

for file in data_dir.glob('*/function_map_new.png'):
    file = str(file)
    file = file.replace('\\', '/')
    res = re.findall("(\w+)/function_map_new.png", file)
    filename = res[0]
    
    img_path = os.path.join(data_dir, filename + '/function_map_new.png')
    canvas_path = os.path.join(data_dir, filename + '/function_map_no_icon.png')
    save_path = os.path.join(data_dir, filename + '/function_map_marked.png')
    
    image = cv2.imread(img_path)
    canvas = cv2.imread(canvas_path)
    font = cv2.FONT_HERSHEY_COMPLEX 

    # Read the image
    height, width, channels = image.shape

    # convert to hsv
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # mask
    lower = np.array([0,0,0])
    upper = np.array([5,5,5])
    mask = cv2.inRange(hsv, lower, upper)

    plan = image.copy()

    # Make the masked areas uniform
    plan[np.where(mask!=0)] = 255
    plan[np.where(mask==0)] = 0

    # Treshold the image and find contours
    _, plan_tresh = cv2.threshold(cv2.cvtColor(plan, cv2.COLOR_BGR2GRAY), 45, 255, cv2.THRESH_BINARY)
    plan_tresh =~ plan_tresh
    cnts, heirarchy = cv2.findContours(plan_tresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    c = max(cnts, key=cv2.contourArea) 

    approx = cv2.approxPolyDP(c, 1, True) 

    # draws boundary of contours. 
    cv2.drawContours(image, [approx], 0, (255, 0, 255), 3)  

    # calculate moments of binary image
    M = cv2.moments(plan_tresh)

    # calculate x,y coordinate of center
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])

    # Used to flatted the array containing 
    # the co-ordinates of the vertices. 
    n = approx.ravel()  
    tuples = []

    # create tuples of the list
    for i,k in zip(n[0::2], n[1::2]):
        tuples.append([i, k])

    i = 0
    topy = 0
    topx = 0
    lowy = width
    lowx = width
    topRight = width
    topLeft = width
    botRight = 0
    botLeft = 0
    leftTop = width
    leftBot = width
    rightTop = 0
    rightBot = 0


    for j in tuples:
        x = j[0]
        y = j[1]

        if(topy < y):
            topy = y
        if(lowy > y):
            lowy = y
        if(topx < x):
            topx = x
        if(lowx > x):
            lowx = x

    for j in tuples:
        x = j[0]
        y = j[1]

        # finding points for top facade
        if(lowx == x):
            if(topLeft > y):
                topLeft = y
        if(topx == x):
            if(topRight > y):
                topRight = y

        # finding points for bot facade
        if(lowx == x):
            if(botLeft < y):
                botLeft = y
        if(topx == x):
            if(botRight < y):
                botRight = y        

        # finding points for left facade
        if(lowy == y):
            if(leftTop > x):
                leftTop = x
        if(topy == y):
            if(leftBot > x):
                leftBot = x   

        # finding points for right facade
        if(lowy == y):
            if(rightTop < x):
                rightTop = x
        if(topy == y):
            if(rightBot < x):
                rightBot = x   



    # paring the values in tuples 
    topLeftT = tuple((lowx, topLeft))
    topRightT = tuple((topx, topRight))
    botLeftT = tuple((lowx, botLeft))
    botRightT = tuple((topx, botRight))
    leftTopT = tuple((leftTop, lowy))
    leftBotT = tuple((leftBot, topy))
    rightTopT = tuple((rightTop, lowy))
    rightBotT = tuple((rightBot, topy))

    # this defines how much the indent of the facade mask is so we only get facade windows.
    indent = 10

    # a mask is created above each of the broken out parts so that we can select it
    # mask for top facade
    topMask = np.zeros((width,height), np.uint8)
    topPts = np.array([[0, 0], [width, 0], topRightT, 
                       [topx, topRight+indent], [lowx, topLeft+indent], topLeftT])
    _=cv2.drawContours(topMask, np.int32([topPts]),0, 255, -1)

    # mask for bot facade
    botMask = np.zeros((width,height), np.uint8)
    botPts = np.array([[0, height], [width, height], botRightT, 
                       [topx, botRight-indent], [lowx, botLeft-indent], botLeftT])
    _=cv2.drawContours(botMask, np.int32([botPts]),0, 255, -1)

    # mask for left facade
    leftMask = np.zeros((width,height), np.uint8)
    leftPts = np.array([[0, 0], [0, height], leftBotT, 
                        [leftBot+indent, topy], [leftTop+indent, lowy], leftTopT])
    _=cv2.drawContours(leftMask, np.int32([leftPts]),0, 255, -1)

    # mask for right facade
    rightMask = np.zeros((width,height), np.uint8)
    rightPts = np.array([[width, height], [width, 0], rightTopT, 
                         [rightTop-indent, lowy], [rightBot-indent, topy], rightBotT])
    _=cv2.drawContours(rightMask, np.int32([rightPts]),0, 255, -1)

    # everything except the mask is now erased from the image
    topImg = image.copy()
    topImg[topMask==0] = 0

    botImg = image.copy()
    botImg[botMask==0] = 0

    leftImg = image.copy()
    leftImg[leftMask==0] = 0

    rightImg = image.copy()
    rightImg[rightMask==0] = 0

    # Rotate images so all facades are facing up
    # Top Image is fine so we ignore it
    botImg = cv2.rotate(botImg, cv2.ROTATE_180)
    leftImg = cv2.rotate(leftImg, cv2.ROTATE_90_CLOCKWISE)
    rightImg = cv2.rotate(rightImg, cv2.ROTATE_90_COUNTERCLOCKWISE)

    # This function contours the windows in each plan, might want to add balconies aswell
    def contourW(img):
        # convert to hsv
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        # mask the windows so they can be countoured and counted
        lower = np.array([0,0,120])
        upper = np.array([5,5,125])
        mask = cv2.inRange(hsv, lower, upper)

        cnt, heirarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        return cnt

    # This function return the x cordinates of the windows on the facade
    def windowPoints(cnt):
        # make it a numpy array
        arr = np.asarray(cnt)

        # get how many windows there is on the facade
        res = re.findall("(\w+),", str(arr.shape))
        loops = int(res[0])

        botx = 5000
        topx = 0
        boty = 5000
        topy = 0
        i = 0
        windowArr = []

        # loop to find the x values of each window
        while i < loops:
            #find highest and lowest x values
            for c in cnt[i]:
                for x,y in c:
                    if(botx > x):
                        botx = x
                    if(topx < x):
                        topx = x
                    if(boty > y):
                        boty = y
                    if(topy < y):
                        topy = y
            if((topx-botx)<=(topy-boty)):
                None
            else:        
                windowArr.append(botx)
                windowArr.append(topx)

            # reset variable between adding each window
            botx = 5000
            topx = 0
            boty = 5000
            topy = 0
            i += 1
        return windowArr 

    # This draws facades on specified side
    def analyseFacade (img, side):
        # print('\nAnalysing ' + side + ' facade')

        # could bake this into the windowPoints function
        windowCnt = contourW(img)
        wArr = windowPoints(windowCnt)

        return int(len(wArr)/2)

    leftNumber = analyseFacade(leftImg, 'left')
    rightNumber = analyseFacade(rightImg, 'right')
    topNumber = analyseFacade(topImg, 'top')
    botNumber = analyseFacade(botImg, 'bot')

    # Go trough each facade and check if there are any windows
    # Then add a marker in the border of each part that has windows
    if(leftNumber == 0):
        None
        # print('Left facade has no openings')
    else:
        # print('Left facade has openings adding marker')
        cv2.rectangle(canvas, (0, 0), (indent, height), (0, 0, 255), -1)

    if(rightNumber == 0):
        None
        # print('Right facade has no openings')
    else:
        # print('Right facade has openings adding marker')
        cv2.rectangle(canvas, (width, 0), (width-indent, height), (0, 0, 255), -1)

    if(topNumber == 0):
        None
        # print('Top facade has no openings')
    else:
        # print('Top facade has openings adding marker')
        cv2.rectangle(canvas, (0, 0), (width, indent), (0, 0, 255), -1)

    if(botNumber == 0):
        None
        # print('Bot facade has no openings')
    else:
        # print('Bot facade has openings adding marker')
        cv2.rectangle(canvas, (0, height), (width, height-indent), (0, 0, 255), -1)

    is_success, plan = cv2.imencode('.png', canvas)

    buf = io.BytesIO(plan)
    buf.seek(0)

    im_plan = PIL.Image.open(buf)
    im_plan.save(save_path)

    buf.close()
    print('Saved ' + filename)
print('\nDone')